In [1]:
%%time
import pandas as pd
from sklearn import metrics,cluster

#### Reading the feature and calculating the pairwise distance between all the points

df = pd.read_csv('gh_lyso_PAAC.txt', sep='\t')# input file which contains features, header=None if no column title
x=df.iloc[:,1:].values
distance=metrics.pairwise.pairwise_distances(x)

#### collecting intra and inter distance for clusters

class clust:
    def __init__(self,ref,comp):
        self.ref=ref
        self.comp=comp
        if self.ref==self.comp:
#             print(f'The intra cluster distance within cluster {self.ref}')
            self.c=1
        else:
#             print(f'The inter cluster distance between cluster {self.ref} and {self.comp}')
            self.c=0
        
    def all_dist(self):
        al_data=[]
        for x1 in range(len(labels)):
            for y1 in range((x1+1)*self.c,len(labels)):
                if labels[x1]==self.ref and labels[y1]==self.comp:
#                     print(f'{x1}..{y1}',distance[x1][y1])
                    al_data.append(distance[x1][y1])
        self.data=al_data
        self.avg=sum(al_data)/len(al_data)
        return self

def un_learn(n):
    kmeans = cluster.KMeans(n_clusters=n,random_state=42) # Number of clusters
    lab = kmeans.fit_predict(x)
    no_l =list(set(lab))
    return lab,no_l

Wall time: 3.49 s


#### To find min and max distance for intra and inter clusters for 10000 iterations

In [24]:
%%time
from collections import Counter
from itertools import combinations
import numpy as np

# a=clust(0,1).all_dist()
# b=clust(0,0).all_dist()
##########################################################################
nn=3 # number of cluster
ni=1 # number of iterations
##########################################################################
lis=[s for s in range(nn)]
grp=list(combinations(lis,2))
grp.extend([(i,i) for i in lis])

min_dis,max_dis,avg_dis={},{},{}
for each in grp:
    min_dis[each],max_dis[each],avg_dis[each]=[],[],[]
    
for each_iter in range(ni):
    print(f'Iteration:{each_iter+1}')
    labels,no_lab=un_learn(nn)
    for j in grp:
        c=clust(j[0],j[1]).all_dist()
        min_dis[j].append(min(c.data))
        max_dis[j].append(max(c.data))
        avg_dis[j].append(c.avg)
temp=[]
for ii in range(ni):
    tem=[]
    for jj in grp[3:]: # only considering intra cluster [(0, 0), (1, 1), (2, 2)]
        tem.append(min(min_dis[jj]))
    temp.append(min(tem))


Iteration:1
Wall time: 86.6 ms


In [22]:
min_dis.keys()dd

dict_keys([(0, 1), (0, 2), (1, 2), (0, 0), (1, 1), (2, 2)])

#### To download the values

In [25]:
out=open('within_min_dist.txt','w')
for iii in temp:
    out.write(str(iii)+'\n')
out.close()   

#### To visualize the data in the table for Average distance clusters

In [49]:
for each_iter in range(1):
    print(f'Iteration:{each_iter+1}')
    labels,no_lab=un_learn(nn)
    label_c=dict(Counter(labels))
    emp_array_avg=np.zeros((len(label_c),len(label_c)))
    emp_array_m=np.zeros((len(label_c),len(label_c)))
    for j in grp:
        c=clust(j[0],j[1]).all_dist()
        emp_array_avg[j[0]][j[1]]=c.avg
        emp_array_avg[j[1]][j[0]]=c.avg
        if j[0]==j[1]:
            emp_array_m[j[0]][j[1]]=max(c.data)
        else:
            emp_array_m[j[0]][j[1]]=min(c.data)
            emp_array_m[j[1]][j[0]]=min(c.data)

pd.DataFrame(emp_array_avg)

Iteration:1


,0,1,2
0,0.021259,0.039074,0.033306
1,0.039074,0.023733,0.031968
2,0.033306,0.031968,0.023998


In [50]:
pd.DataFrame(emp_array_m)

,0,1,2
0,0.037433,0.025910,0.019056
1,0.025910,0.042642,0.022109
2,0.019056,0.022109,0.037556


In [13]:
grp[3:]

[(0, 0), (1, 1), (2, 2)]